In [1]:
from dotenv import load_dotenv
import os
load_dotenv() 

True

In [3]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [4]:
from datetime import datetime
import pandas as pd
from src.data_utils import load_and_process_taxi_data

from_year = 2022
# to_year = datetime.now().year
to_year = 2023
print(f"Download raw data from {from_year} to {to_year}")

rides = pd.DataFrame()
chunks = []
for year in range(from_year, to_year+1):

    rides_one_year = load_and_process_taxi_data(year)

    chunks.append(rides_one_year)
    break

# Concatenate all chunks at the end
rides = pd.concat(chunks, ignore_index=True)
print("Data loading complete.")

Download raw data from 2022 to 2023
File already exists for 2022-01.
Loading data for 2022-01...
Total records: 2,463,931
Valid records: 2,415,141
Records dropped: 48,790 (1.98%)
Successfully processed data for 2022-01.
File already exists for 2022-02.
Loading data for 2022-02...
Total records: 2,979,431
Valid records: 2,921,118
Records dropped: 58,313 (1.96%)
Successfully processed data for 2022-02.
File already exists for 2022-03.
Loading data for 2022-03...
Total records: 3,627,882
Valid records: 3,551,986
Records dropped: 75,896 (2.09%)
Successfully processed data for 2022-03.
File already exists for 2022-04.
Loading data for 2022-04...
Total records: 3,599,920
Valid records: 3,522,113
Records dropped: 77,807 (2.16%)
Successfully processed data for 2022-04.
File already exists for 2022-05.
Loading data for 2022-05...
Total records: 3,588,295
Valid records: 3,509,056
Records dropped: 79,239 (2.21%)
Successfully processed data for 2022-05.
File already exists for 2022-06.
Loading dat

In [5]:
rides

,pickup_datetime,pickup_location_id
0,2022-01-01 00:35:40,142
1,2022-01-01 00:33:43,236
2,2022-01-01 00:53:21,166
3,2022-01-01 00:25:21,114
4,2022-01-01 00:36:48,68
...,...,...
38733116,2022-12-31 23:46:00,16
38733117,2022-12-31 23:13:24,75
38733118,2022-12-31 23:00:49,168
38733119,2022-12-31 23:02:50,238


In [6]:
rides.shape

(38733121, 2)

In [7]:
from src.data_utils import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

In [8]:
ts_data.shape

(2268840, 3)

In [9]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2268840 entries, 0 to 2268839
Data columns (total 3 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pickup_hour         datetime64[ns]
 1   pickup_location_id  int16         
 2   rides               int16         
dtypes: datetime64[ns](1), int16(2)
memory usage: 26.0 MB


## Updating pickup_hour time zone to NYC/EST

In [ ]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize("EST")

In [11]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2268840 entries, 0 to 2268839
Data columns (total 3 columns):
 #   Column              Dtype              
---  ------              -----              
 0   pickup_hour         datetime64[ns, EST]
 1   pickup_location_id  int16              
 2   rides               int16              
dtypes: datetime64[ns, EST](1), int16(2)
memory usage: 26.0 MB


In [12]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2022-01-01 00:00:00-05:00,2,0
1,2022-01-01 01:00:00-05:00,2,0
2,2022-01-01 02:00:00-05:00,2,0
3,2022-01-01 03:00:00-05:00,2,0
4,2022-01-01 04:00:00-05:00,2,0
...,...,...,...
2268835,2022-12-31 19:00:00-05:00,263,222
2268836,2022-12-31 20:00:00-05:00,263,224
2268837,2022-12-31 21:00:00-05:00,263,223
2268838,2022-12-31 22:00:00-05:00,263,143


In [14]:
import hopsworks

api_key = os.getenv('HOPSWORKS_API_KEY')  
project_name = os.getenv('HOPSWORKS_PROJECT_NAME')  

# pip install confluent-kafka
# Initialize connection to Hopsworks  
project = hopsworks.login(  
    api_key_value=api_key,  
    project=project_name  
)  
print(f"Successfully connected to Hopsworks project: {project_name}")

2025-03-03 15:53:55,883 INFO: Initializing external client
2025-03-03 15:53:55,883 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 15:53:56,693 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215642
Successfully connected to Hopsworks project: nyc_taxi_cda500


In [15]:
feature_store = project.get_feature_store()

In [22]:
FEATURE_GROUP_NAME = "time_series_hourly_feature_group"
FEATURE_GROUP_VERSION = 1

In [23]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time series data aggregated by hour",
    primary_key=["pickup_location_id","pickup_hour"],
    event_time="pickup_hour"
)

In [24]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1215642/fs/1202294/fg/1401901


Uploading Dataframe: 100.00% |██████████| Rows 2268840/2268840 | Elapsed Time: 03:01 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1215642/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)

In [ ]:
df_memory_mb = rides.memory_usage(deep=True).sum() / (1024 * 1024)  
print(f"DataFrame size: {df_memory_mb:.2f} MB")

In [28]:
ts_data.head()

,pickup_hour,pickup_location_id,rides
0,2022-01-01 00:00:00-05:00,2,0
1,2022-01-01 01:00:00-05:00,2,0
2,2022-01-01 02:00:00-05:00,2,0
3,2022-01-01 03:00:00-05:00,2,0
4,2022-01-01 04:00:00-05:00,2,0


In [27]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2268840 entries, 0 to 2268839
Data columns (total 3 columns):
 #   Column              Dtype              
---  ------              -----              
 0   pickup_hour         datetime64[ns, EST]
 1   pickup_location_id  int16              
 2   rides               int16              
dtypes: datetime64[ns, EST](1), int16(2)
memory usage: 26.0 MB
